Before running the code, please go through the README file to understand it

Note:
1.Please run the notebook on a cluster with UC enabled.


Note: 2.The UC cluster does not support the No Isolation Shared mode; therefore, we changed the cluster to Single User Access mode

In [0]:
#Install Pyyaml library
%pip install pyyaml

In [0]:
#Import the library
import yaml

In [0]:
with open("/Workspace/Users/mahesh.pasalapudi@digivatelabs.com/cluster_migration_updated/config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [0]:
#Extract configuration values from the loaded YAML file
DATABRICKS_INSTANCE =config.get("DATABRICKS_INSTANCE")
TOKEN = config.get("TOKEN")
new_runtime_version = config.get("new_runtime_version")
cluster_id = config.get("cluster_id")
cluster_ids_to_revert = config.get("cluster_ids_to_revert")
my_volume_path = config.get("my_volume_path")
cluster_ids = config.get("cluster_ids")
single_user_name = config.get("single_user_name")   

Before start the processs store the cluster details for backup, this json files store in same working directory

In [0]:
import os
import json
import requests

CLUSTER_LIST_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/list"
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

def save_cluster_details_to_file(cluster_id, cluster_details):
    """Save the current cluster details to a JSON file."""
    file_name = f"cluster_backup_{cluster_id}.json"# want to store in volume replace the path with volume path
    with open(file_name, "w") as file:
        json.dump(cluster_details, file, indent=4)
    print(f"Cluster details for {cluster_id} saved to {file_name}")

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json() 
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def get_cluster_list():
    """Fetch the list of clusters."""
    response = requests.get(CLUSTER_LIST_URL, headers=HEADERS)
    if response.status_code == 200:
        clusters = response.json().get("clusters", [])
        return [cluster['cluster_id'] for cluster in clusters]
    else:
        print(f"Failed to get cluster list: {response.text}")
        return []

# Fetch the list of cluster IDs
cluster_ids = get_cluster_list()

# Fetch the details of each cluster and save them
for cluster_id in cluster_ids:
    cluster_details = get_cluster_details(cluster_id)
    if cluster_details:
        save_cluster_details_to_file(cluster_id, cluster_details)


Cluster Validation script

In [0]:
import requests
import json

# Define your Databricks instance URL and token
# DATABRICKS_INSTANCE = "<Your Databricks Instance URL>"
# TOKEN = "<Your Databricks API Token>"

# Define API URLs
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def check_cluster_details(cluster_id):
    """Check all available cluster details."""
    cluster_details = get_cluster_details(cluster_id)

    if cluster_details:
        # Print the entire cluster details for inspection
        print("Cluster Details:")
        print(json.dumps(cluster_details, indent=4))  
        
        # Check Unity Catalog (looking for data_security_mode, single user mode indicates Unity Catalog)
        unity_catalog_status = cluster_details.get('data_security_mode', 'Not Available')
        
        if unity_catalog_status == 'SINGLE_USER':
            print("Unity Catalog is enabled on the cluster.")
        else:
            print(f"Unity Catalog status: {unity_catalog_status}")
        
        # Check Photon enabled (check if it's part of the spark version or configuration)
        spark_version = cluster_details.get('spark_version', 'Not Available')
        if "photon" in spark_version.lower():
            print("Photon is enabled for this cluster.")
        else:
            print("Photon is not enabled for this cluster.")
        
        # Check Runtime Version
        runtime_version = cluster_details.get('spark_version', 'Not Available')
        print(f"Runtime Version: {runtime_version}")

        # Check Environment Variables
        spark_env_vars = cluster_details.get('spark_env_vars', {})
        if spark_env_vars:
            print("Environment Variables:")
            for var, value in spark_env_vars.items():
                print(f"  {var}: {value}")
        else:
            print("No environment variables set on the cluster.")
        
        # Check Termination Time (if the cluster is terminated)
        termination_time = cluster_details.get('terminated_time', 'Cluster still running or not available')
        print(f"Termination Time: {termination_time}")
        
        # Check if the cluster is active or terminated
        state = cluster_details.get('state', 'Not Available')
        print(f"Cluster State: {state}")
        
    else:
        print("Failed to fetch cluster details for validation.")

if __name__ == "__main__":
    # cluster_id = "0108-050342-xm8nm1ea"  
    # Check all cluster details
    check_cluster_details(cluster_id)


##### Update Databricks Cluster Runtime and Enable Unity Catalog

In [0]:
import requests
import json
import time

# API Endpoints
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
CLUSTER_EDIT_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/edit"
CLUSTER_RESTART_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/restart"

HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def restart_cluster(cluster_id):
    """Restart the Databricks cluster."""
    restart_payload = {
        "cluster_id": cluster_id
    }
    response = requests.post(CLUSTER_RESTART_URL, headers=HEADERS, json=restart_payload)
    if response.status_code == 200:
        print(f"Cluster {cluster_id} restart requested successfully.")
        return True
    else:
        print(f"Failed to restart cluster: {response.text}")
        return False

def enable_unity_catalog_on_cluster(cluster_id, new_runtime_version):
    """Enable Unity Catalog on a Databricks cluster by updating its configuration."""
    # Fetch current cluster details
    cluster_details = get_cluster_details(cluster_id)

    if cluster_details:
        # Ensure the correct runtime version
        cluster_details['spark_version'] = new_runtime_version

        # Remove conflicting Spark configurations
        spark_conf = cluster_details.get("spark_conf", {})
        if "spark.databricks.passthrough.enabled" in spark_conf:
            print("Removing conflicting spark.databricks.passthrough.enabled configuration.")
            del spark_conf["spark.databricks.passthrough.enabled"]

        # Prepare the payload to update the cluster configuration
        payload = {
            "cluster_id": cluster_id,
            "cluster_name": cluster_details.get("cluster_name", "Default Cluster"),
            "spark_version": new_runtime_version,  
            "node_type_id": cluster_details["node_type_id"], 
            "driver_node_type_id": cluster_details["driver_node_type_id"],  
            "num_workers": cluster_details.get("num_workers"),
            "autoscale" : cluster_details.get("autoscale"),  
            "autotermination_minutes": cluster_details.get("autotermination_minutes"), 
            "spark_conf": spark_conf,  
            "aws_attributes": cluster_details.get("aws_attributes", {}), 
            "custom_tags": cluster_details.get("custom_tags", {}), 
            "enable_elastic_disk": cluster_details.get("enable_elastic_disk", False),
            "init_scripts": cluster_details.get("init_scripts", []),  
            "single_user_name": single_user_name, 
            "data_security_mode": "SINGLE_USER", 
            "runtime_engine": cluster_details.get("runtime_engine", "STANDARD"),  
        }

        # Send the update request to Databricks API to update the cluster
        response = requests.post(CLUSTER_EDIT_URL, headers=HEADERS, json=payload)

        if response.status_code == 200:
            print(f"Cluster updated successfully with runtime version: {new_runtime_version}")
            return True
        else:
            print(f"Failed to update cluster: {response.text}")
            return False
    else:
        print("Unable to fetch cluster details for update.")
        return False

def update_cluster_for_unity_catalog(cluster_id, new_runtime_version):
    """Main function to enable Unity Catalog on a Databricks cluster."""
    # Enable Unity Catalog on the cluster
    if enable_unity_catalog_on_cluster(cluster_id, new_runtime_version):
        # If the update was successful, wait for the change to take effect and restart the cluster
        print("Cluster successfully updated for Unity Catalog. Restarting...")
        # Allow time for the cluster to apply updates (could be customized depending on load)
        time.sleep(10)  
        
        if restart_cluster(cluster_id):
            print("Cluster successfully restarted.")
        else:
            print("Failed to restart the cluster.")
    else:
        print("Failed to enable Unity Catalog on the cluster.")

# Example Execution
if __name__ == "__main__":
    update_cluster_for_unity_catalog(cluster_id, new_runtime_version)


###### Upgrade Databricks Cluster Runtime, Enable Unity Catalog, and Migrate Environment Variables


In [0]:
import requests
import json

# Databricks API Endpoint
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

# API Endpoints
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
CLUSTER_EDIT_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/edit"

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def upgrade_cluster(cluster_id, new_runtime_version, single_user_name):
    """Upgrade the cluster runtime version and modify other configurations, including environment variables."""
    # Fetch current cluster details
    cluster_details = get_cluster_details(cluster_id)

    if cluster_details:
        # Extract environment variables (spark_env_vars) before update
        spark_env_vars = cluster_details.get("spark_env_vars", {}) 

        # Modify the runtime version in the cluster configuration
        cluster_details['spark_version'] = new_runtime_version

        # Prepare the payload to update the cluster
        payload = {
            "cluster_id": cluster_id,
            "cluster_name": cluster_details.get("cluster_name", "Default Cluster"), 
            "spark_version": cluster_details["spark_version"], 
            "node_type_id": cluster_details["node_type_id"],  
            "driver_node_type_id": cluster_details["driver_node_type_id"],  
            "num_workers": cluster_details.get("num_workers"), 
            "autoscale" : cluster_details.get("autoscale"),
            "autotermination_minutes": cluster_details.get("autotermination_minutes"),  
            "spark_conf": cluster_details.get("spark_conf", {}),  
            "aws_attributes": cluster_details.get("aws_attributes", {}), 
            "custom_tags": cluster_details.get("custom_tags", {}),  
            "enable_elastic_disk": cluster_details.get("enable_elastic_disk", False), 
            "init_scripts": cluster_details.get("init_scripts", []),  
            "single_user_name": single_user_name,  
            "data_security_mode": "SINGLE_USER",  
            "runtime_engine": cluster_details.get("runtime_engine", "STANDARD"),  
            "spark_env_vars": spark_env_vars, 
            "unity_catalog_enabled": True  
        }

        # Send the update request to Databricks API
        response = requests.post(CLUSTER_EDIT_URL, headers=HEADERS, json=payload)

        # Log the response to see if there are any issues
        if response.status_code == 200:
            print(f"Cluster updated successfully with runtime version: {new_runtime_version}")
            print(f"Environment variables before update: {spark_env_vars}")
            return True
        else:
            # Print full response to help debug the issue
            print(f"Failed to update cluster: {response.text}")
            print(f"Response code: {response.status_code}")
            print(f"Response content: {response.content}")
            return False
    else:
        print("Unable to fetch cluster details for update.")
        return False

if __name__ == "__main__":
    if upgrade_cluster(cluster_id, new_runtime_version, single_user_name):
        print("Cluster upgraded and environment variables updated successfully.")
    else:
        print("Failed to upgrade the cluster.")


Update the multiple clusters at a time


In [0]:
import requests
import json

# Databricks API Endpoint
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

# API Endpoints
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
CLUSTER_EDIT_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/edit"

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def upgrade_cluster(cluster_id, new_runtime_version, single_user_name):
    """Upgrade the cluster runtime version and modify other configurations, including environment variables."""
    # Fetch current cluster details
    cluster_details = get_cluster_details(cluster_id)

    if cluster_details:
        # Extract environment variables (spark_env_vars) before update
        spark_env_vars = cluster_details.get("spark_env_vars", {})  

        # Modify the runtime version in the cluster configuration
        cluster_details['spark_version'] = new_runtime_version

        # Prepare the payload to update the cluster
        payload = {
            "cluster_id": cluster_id,
            "cluster_name": cluster_details.get("cluster_name", "Default Cluster"),  
            "spark_version": cluster_details["spark_version"],  
            "node_type_id": cluster_details["node_type_id"], 
            "driver_node_type_id": cluster_details["driver_node_type_id"],  
            "num_workers": cluster_details.get("num_workers"),  
            "autoscale" : cluster_details.get("autoscale"),
            "autotermination_minutes": cluster_details.get("autotermination_minutes"),  
            "spark_conf": cluster_details.get("spark_conf", {}),  
            "aws_attributes": cluster_details.get("aws_attributes", {}),  
            "custom_tags": cluster_details.get("custom_tags", {}),  
            "enable_elastic_disk": cluster_details.get("enable_elastic_disk", False),  
            "init_scripts": cluster_details.get("init_scripts", []),  
            "single_user_name": single_user_name,  
            "data_security_mode": "SINGLE_USER",  
            "runtime_engine": cluster_details.get("runtime_engine", "STANDARD"),  
            "spark_env_vars": spark_env_vars,  
            "unity_catalog_enabled": True  
        }

        # Send the update request to Databricks API
        response = requests.post(CLUSTER_EDIT_URL, headers=HEADERS, json=payload)

        # Log the response to see if there are any issues
        if response.status_code == 200:
            print(f"Cluster {cluster_id} updated successfully with runtime version: {new_runtime_version}")
            print(f"Environment variables before update: {spark_env_vars}")
            return True
        else:
            # Print full response to help debug the issue
            print(f"Failed to update cluster {cluster_id}: {response.text}")
            print(f"Response code: {response.status_code}")
            print(f"Response content: {response.content}")
            return False
    else:
        print(f"Unable to fetch cluster details for {cluster_id} for update.")
        return False

def upgrade_multiple_clusters(cluster_ids, new_runtime_version, single_user_name):
    """Upgrade multiple clusters by iterating through a list of cluster IDs."""
    for cluster_id in cluster_ids:
        print(f"Upgrading cluster: {cluster_id}")
        if upgrade_cluster(cluster_id, new_runtime_version, single_user_name):
            print(f"Cluster {cluster_id} upgraded successfully.")
        else:
            print(f"Failed to upgrade cluster {cluster_id}.")

if __name__ == "__main__":
    upgrade_multiple_clusters(cluster_ids, new_runtime_version, single_user_name)


##### Creating an External Volume in Databricks (optional)

In [0]:
%sql
CREATE EXTERNAL VOLUME my_volume
LOCATION 's3://aws-glue11/my_volume_path/';

##### Displaying Volumes

In [0]:
%sql
show volumes;

##### Python Script for Upgrading a Databricks Cluster and Adding Environment Variable "my_volume_path"


In [0]:
import requests
import json
# Databricks API Endpoint
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

# API Endpoints
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
CLUSTER_EDIT_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/edit"

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def upgrade_cluster(cluster_id, new_runtime_version, single_user_name, my_volume_path):
    """Upgrade the cluster runtime version and modify other configurations, including environment variables."""
    # Fetch current cluster details
    cluster_details = get_cluster_details(cluster_id)

    if cluster_details:
        # Extract environment variables (spark_env_vars) before update
        spark_env_vars = cluster_details.get("spark_env_vars", {})  

        # Add the new environment variable for volume path
        spark_env_vars["MY_VOLUME_PATH"] = my_volume_path  

        # Modify the runtime version in the cluster configuration
        cluster_details['spark_version'] = new_runtime_version

        # Prepare the payload to update the cluster
        payload = {
            "cluster_id": cluster_id,
            "cluster_name": cluster_details.get("cluster_name", "Default Cluster"),  
            "spark_version": cluster_details["spark_version"],  
            "node_type_id": cluster_details["node_type_id"],  
            "driver_node_type_id": cluster_details["driver_node_type_id"],  
            "num_workers": cluster_details.get("num_workers"),
            "autoscale" : cluster_details.get("autoscale"),  
            "autotermination_minutes": cluster_details.get("autotermination_minutes"), 
            "spark_conf": cluster_details.get("spark_conf", {}),  
            "aws_attributes": cluster_details.get("aws_attributes", {}), 
            "custom_tags": cluster_details.get("custom_tags", {}),  
            "enable_elastic_disk": cluster_details.get("enable_elastic_disk", False),  
            "init_scripts": cluster_details.get("init_scripts", []),  
            "single_user_name": single_user_name,  
            "data_security_mode": "SINGLE_USER",  
            "runtime_engine": cluster_details.get("runtime_engine", "STANDARD"),  
            "spark_env_vars": spark_env_vars,  
            "unity_catalog_enabled": True  
        }

        # Send the update request to Databricks API
        response = requests.post(CLUSTER_EDIT_URL, headers=HEADERS, json=payload)

        # Log the response to see if there are any issues
        if response.status_code == 200:
            print(f"Cluster updated successfully with runtime version: {new_runtime_version}")
            print(f"Environment variables before update: {spark_env_vars}")
            return True
        else:
            # Print full response to help debug the issue
            print(f"Failed to update cluster: {response.text}")
            print(f"Response code: {response.status_code}")
            print(f"Response content: {response.content}")
            return False
    else:
        print("Unable to fetch cluster details for update.")
        return False

if __name__ == "__main__":
     upgrade_cluster(cluster_id, new_runtime_version, single_user_name, my_volume_path)


Data Reconciliation for Cluster

In [0]:
import os
import json
import requests

# Databricks API Endpoint
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

def load_cluster_backup(cluster_id):
    """Load the backup file for the cluster from the mounted volume."""
    backup_path = f"{my_volume_path}/cluster_backup_{cluster_id}.json"
    if os.path.exists(backup_path):
        with open(backup_path, "r") as file:
            return json.load(file)
    else:
        print(f"No backup found for cluster {cluster_id}")
        return None

def get_cluster_details(cluster_id):
    """Fetch the current cluster details and return them as a dictionary."""
    response = requests.get(CLUSTER_GET_URL, headers=HEADERS, params={"cluster_id": cluster_id})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to get cluster details: {response.text}")
        return None

def reconcile_cluster_details(current_details, backup_details):
    """Compare current cluster details with backup (focusing on key fields)."""
    # Define the keys we want to compare
    keys_to_compare = [
        'cluster_name', 
        'spark_env_vars', 
        'autotermination_minutes', 
        'photon', 
        'spark_version'
    ]
    
    discrepancies = []

    # Compare the selected keys between current and backup details
    for key in keys_to_compare:
        current_value = current_details.get(key)
        backup_value = backup_details.get(key)
        
        if current_value != backup_value:
            discrepancies.append({
                "key": key,
                "current_value": current_value,
                "backup_value": backup_value
            })

    return discrepancies

def print_reconciliation_output(cluster_id, current_details, discrepancies):
    """Print a detailed and user-friendly reconciliation report."""
    print(f"\n--- Reconciliation Report for Cluster {cluster_id} ---\n")

    if discrepancies:
        print("Discrepancies found between current cluster and backup:\n")
        for discrepancy in discrepancies:
            print(f"Field: {discrepancy['key']}")
            print(f"  Current Value: {discrepancy['current_value']}")
            print(f"  Backup Value: {discrepancy['backup_value']}")
            print("-" * 50)
    else:
        print("No discrepancies found.")

def check_unity_catalog(current_details, backup_details):
    """Check Unity Catalog status in both current and backup details."""
    current_data_security_mode = current_details.get('data_security_mode', 'Not Available')
    backup_data_security_mode = backup_details.get('data_security_mode', 'Not Available')

    print("\n--- Unity Catalog Check ---")
    if current_data_security_mode != backup_data_security_mode:
        print(f"Discrepancy in Unity Catalog:")
        print(f"  Current Data Security Mode: {current_data_security_mode}")
        print(f"  Backup Data Security Mode: {backup_data_security_mode}")
    else:
        print(f"Unity Catalog status is the same in both current and backup (Data Security Mode: {current_data_security_mode})")


    print("\n--- Cluster Summary ---")
    # Summary of the cluster update
    current_runtime_version = current_details.get('spark_version', 'Not Available')
    environment_variables_before = current_details.get('spark_env_vars', {})
    
    print(f"\nCluster updated successfully with runtime version: {current_runtime_version}")
    print(f"Environment variables before update: {json.dumps(environment_variables_before, indent=4)}")
    print(f"Autotermination minutes: {current_details.get('autotermination_minutes', 'Not Available')}")
    print(f"Photon: {current_details.get('photon', 'Not Available')}")

    print("\nCluster reconciliation completed successfully.\n")


def perform_data_reconciliation(cluster_id):
    """Perform reconciliation for a single cluster."""
    # Load the current cluster details
    current_cluster_details = get_cluster_details(cluster_id)
    
    if current_cluster_details:
        # Load the backup of the cluster details
        backup_cluster_details = load_cluster_backup(cluster_id)
        
        if backup_cluster_details:
            # Perform reconciliation between the current and backup details
            discrepancies = reconcile_cluster_details(current_cluster_details, backup_cluster_details)
            # Print reconciliation report, passing current cluster details to the summary function
            print_reconciliation_output(cluster_id, current_cluster_details, discrepancies)
        else:
            print(f"Skipping reconciliation for cluster {cluster_id} due to missing backup.")
    else:
        print(f"Skipping reconciliation for cluster {cluster_id} due to missing current details.")


# Example: Perform reconciliation for a specific cluster
# cluster_id = "your-cluster-id-here" 
perform_data_reconciliation(cluster_id)


Backup your cluster with help below mentioned code

This script is designed to revert Databricks clusters to their previous state based on a backup configuration. The script fetches the current cluster details and compares them with a backup configuration (stored in a JSON file), then reverts the cluster to its previous state by updating its configuration.

In [0]:
import os
import json
import requests

# Databricks API Endpoint
CLUSTER_EDIT_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/edit"
CLUSTER_GET_URL = f"{DATABRICKS_INSTANCE}/api/2.0/clusters/get"
HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

def load_cluster_details_from_file(cluster_id):
    """Load the saved cluster details from the JSON file."""
    file_name = f"cluster_backup_{cluster_id}.json" # if file in volume replace with volume path
    if os.path.exists(file_name):
        with open(file_name, "r") as file:
            return json.load(file)
    else:
        print(f"Backup file for cluster {cluster_id} not found.")
        return None

def get_cluster_details(cluster_id):
    """Retrieve the current details of a cluster."""
    response = requests.get(f"{CLUSTER_GET_URL}?cluster_id={cluster_id}", headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve details for cluster {cluster_id}: {response.text}")
        return None

def revert_cluster_to_previous_state(cluster_id):
    """Revert the cluster to its previous state using the backup configuration."""
    # Get the current cluster details
    cluster_details = get_cluster_details(cluster_id)
    
    if cluster_details:
        # Check if Unity Catalog is enabled, skip if it's already enabled
        if cluster_details.get("unity_catalog_enabled", False):
            print(f"Cluster {cluster_id} already has Unity Catalog enabled. Skipping update.")
            return False  # Skip the update if Unity Catalog is enabled
        
        # Load the backup configuration from the file
        backup_details = load_cluster_details_from_file(cluster_id)
        
        if backup_details:
            # Prepare the payload to update the cluster to its original state
            payload = {
                "cluster_id": cluster_id,
                "cluster_name": backup_details.get("cluster_name", "Default Cluster"),
                "spark_version": backup_details.get("spark_version", ""),
                "node_type_id": backup_details.get("node_type_id", ""),
                "driver_node_type_id": backup_details.get("driver_node_type_id", ""),
                "num_workers": backup_details.get("num_workers"),
                "autoscale": backup_details.get("autoscale"),
                "autotermination_minutes": backup_details.get("autotermination_minutes"),
                "spark_conf": backup_details.get("spark_conf", {}),
                "aws_attributes": backup_details.get("aws_attributes", {}),
                "custom_tags": backup_details.get("custom_tags", {}),
                "enable_elastic_disk": backup_details.get("enable_elastic_disk", False),
                "init_scripts": backup_details.get("init_scripts", []),
                "single_user_name": backup_details.get("single_user_name", ""),
                "data_security_mode": backup_details.get("data_security_mode", "SINGLE_USER"),
                "runtime_engine": backup_details.get("runtime_engine", "STANDARD"),
                "spark_env_vars": backup_details.get("spark_env_vars", {}),
                "unity_catalog_enabled": backup_details.get("unity_catalog_enabled", False)  
            }

            # Send the update request to Databricks API
            response = requests.post(CLUSTER_EDIT_URL, headers=HEADERS, json=payload)

            # Log the response to see if there are any issues
            if response.status_code == 200:
                print(f"Cluster {cluster_id} reverted to its previous state successfully.")
                return True
            else:
                print(f"Failed to revert cluster {cluster_id}: {response.text}")
                return False
        else:
            print(f"Unable to load backup details for cluster {cluster_id}.")
            return False
    else:
        print(f"Unable to retrieve details for cluster {cluster_id}.")
        return False

def revert_multiple_clusters(cluster_ids):
    """Revert multiple clusters to their previous states by iterating through a list of cluster IDs."""
    for cluster_id in cluster_ids:
        print(f"Reverting cluster: {cluster_id}")
        if revert_cluster_to_previous_state(cluster_id):
            print(f"Cluster {cluster_id} reverted successfully.")
        else:
            print(f"Failed to revert cluster {cluster_id}.")

if __name__ == "__main__":  
    # Revert clusters
    if cluster_ids_to_revert:
        print(f"Reverting {len(cluster_ids_to_revert)} clusters to their previous state.")
        revert_multiple_clusters(cluster_ids_to_revert)
    else:
        print("No cluster IDs provided for reverting.")
